In [ ]:
import requests
import re
from bs4 import BeautifulSoup as bs
from requests_html import HTMLSession
import json
import time
import pandas as pd
import os
import traceback
import sys
import csv

In [ ]:
# This file is for scraping the relevant data from each link

In [ ]:
df=pd.DataFrame(columns=['case title', 'judges name(s)', 'date of judgment', 'citation','issues','decision','cited cases'])
s=HTMLSession()
f=open('SC_links.txt','r')
no_of_links=0
for link in f:
    doc=s.get(url=link)
    soup=bs(doc.content,'html.parser')
    try:
        check_title=soup.find('title')
        name=check_title.text
        cut=re.findall('on [0-9].+',check_title.text)      #searches for the date in check_title
        case_title=name.replace(cut[0],'').strip()         #case_title value
        case_date=cut[0].replace('on','').strip()          #case_date value
    except Exception:
        print('title error')
        print(traceback.format_exc())
        case_title=''
        case_date=''
    bench_class=soup.find('div',{'class':'doc_bench'})
    if bench_class!=None:
        judges_names=bench_class.text.replace('Bench:','').strip() # if bench_class exists, pulls judges names from it
    else:
        print('enters judge else')     # if bench_class does not exist, searches for judges names from the bottom of the document
        bench_h=''
        try:
            temp_=soup.find_all('p')
            for i in range(-1,-len(temp_),-1):
                if i>=-3:
                    bench_h+=temp_[i].text
                else:
                    if '....J' in temp_[i].text:    # Most supreme court docs will have '.....J' at the end along with the judges names
                        bench_h+=temp_[i].text
                    else:
                        break    
            bench_h=bench_h.replace('\n',' ')
            li_5=str.split(bench_h)
            bench_h=' '.join(li_5)
            extr=re.findall('[\(|\[].+?[\)|\]]',bench_h)   #Usually judges names are enclosed in paranthesis or square brackets at the end of the document
            final_bench=''
            for i in extr:
                judges_names+=i+', '   #
                judges_names=judges_names.strip()
        except:
            print('Bench Error')
            print(traceback.format_exc())
            judges_names=''
    check_citation=soup.find('div',{'class':'doc_citations'})
    if check_citation!=None:
        citation=check_citation.text.lower().replace('equivalent citations:','').upper()   #citation value is taken from the class doc_citations
    else:
        print('enters citation else')    #if doc citation class is not present we try a couple of things to get the citation
        try:
            pre_cit=soup.find(id='pre_1')
            pre_cit=pre_cit.replace('\n',' ')
            temp_cit=str.split(pre_cit)
            pre_cit=' '.join(temp_cit)
            pre_cit=re.findall('supreme court of india.+?[0-9].+?[of|0f] ....',pre_cit.lower()) #search for citation in pre_1 class 
            pre_cit=str.split(fes[0].lower(),'jurisdiction')
            if pre_cit!=[]:
                citation=pre_cit[-1].upper().strip()
            else:
                print('enters citation else #2')
                final_cit=re.findall('[no|n0].+? [0-9]+ of [0-9]+',pre_cit.lower())   #search for case no if the above does not work
                final_cit=str.split(final_cit[0],' ')
                citation=''
                for i in range (1,len(final_cit)):
                    citation+=final_cit[i]+' '   #citation value is assigned
                citation=citation.strip()
        except:
            print('Citation Error')
            print(traceback.format_exc())
            citation=''
            
    #To find issues, we search if an <a> has any of the words in list_words, and we check so that duplicates and incomplete phrases do not get added to the hold_issue list            
    count=0
    hold_issue=[]
    cited_case=[]
    check_a=soup.find_all('a')
    list_words=['section','rule','order','article']
    for i in check_a:
        wash=i.text.split()
        i=''
        for word in wash:
            i+=word+' '
        for k in list_words:
            if k in i.lower():
                for j in hold_issue:
                    try:
                        sec=re.findall(k+' [0-9]+ [a-z].',i.lower())   #We check the completeness of the text. e.g: So that "Section 27 of the Contract Act" is accepted into the list and "Section 12" is not accepted
                        if sec[0] in j.lower().strip():
                            count=1
                            break
                    except:
                        pass
                    if i.lower() in j.lower().strip():
                        count=1
                        break
                    else:
                        count=0
        if count==0:
            for k in list_words:
                if k in i.lower():
                    try:
                        final_check=re.findall(k+' [0-9]+ [a-z].',i.lower())
                        if final_check[0]!=[]:
                            hold_issue.append(i)
                    except:
                        pass
        # We do a similar process for the cited_cases, we check for duplicates or incomplete
        if ' v. ' in i.lower():
            cite_val=0
            for case_check in cited_case:
                if i.lower() in case_check.lower():
                    cite_val=1
                    break
            if cite_val==0:
                temp_cited_case=i.lower().replace('\n',' ')
                temp_cited_case=i.lower().replace('\t',' ')
                temp_cited_case=temp_cited_case.strip()
                temp_cited_list=temp_cited_case.split()
                temp_cited_string=''
                for temp in temp_cited_list:
                    temp_cited_string+=temp+' '
                cited_case.append(temp_cited_string)

    
    len_cited_case=len(cited_case)
    step=0
    cited_dict={}                          # Counts the no. of cited_cases and assigns score such that, the cases that appear lower in the judgment will have a high value; Definitely need to work on this
    for j in cited_case:
        if len_cited_case<=3:
            cited_dict[j]=1.0
        else:
            z=len_cited_case/3
            if z>=step:
                cited_dict[j]=0.8
            else:
                cited_dict[j]=1.0
        step+=1
    issues=''
    for issue in hold_issue:
        issues+=issue+'; '
    try:
        #To get the decision, we check for words in dec_word_list and check if they occur somewhat close to words in rej_list and acc_list (either in front of them or behind them)
        dec_check=soup.find_all('p')
        got_dec=0
        for dec_iter in range(-1,-len(dec_check),-1):
            dec_word_list=['petitions','petition','appeals','appeal','suits','special leave petitions','S.L.P']
            for dec_word in dec_word_list:
                if dec_word in dec_check[dec_iter].text.lower():
                    dec_capture=re.findall(dec_word+'.+',dec_check[dec_iter].text.lower())
                    if len(dec_capture)==0:
                        break
                        
                    dec_capture=dec_capture[0].split()
                    try:
                        dec_word_index=dec_capture.index(dec_word)
                    except:
                        break
                    rej_list=['fail','dismiss','dispose']
                    acc_list=['success','allow','accept','correct','set aside']
                    dec_capture_len=len(dec_capture)
                    if dec_capture_len>8:
                        dec_list_back=dec_capture[:dec_word_index]
                        dec_list_forward=dec_capture[dec_word_index+1:dec_word_index+8]
                        if dec_list_back!=[]:
                            for past_word in dec_list_back:
                                for rej in rej_list:
                                    if rej in past_word:
                                        decision=0
                                        got_dec=1
                                        break
                                for acc in acc_list:
                                    if got_dec==0:
                                        if acc in past_word:
                                            decision=1
                                            got_dec=1
                                            break
                                    else:
                                        break
                                if got_dec==1:
                                    break
                        if dec_list_forward!=[]:
                            for for_word in dec_list_forward:
                                for rej in rej_list:
                                    if rej in for_word:
                                        decision=0
                                        got_dec=1
                                        break
                                for acc in acc_list:
                                    if got_dec==0:
                                        if acc in for_word:
                                            decision=1
                                            got_dec=1
                                            break
                                    else:
                                        break
                                if got_dec==1:
                                    break

                    else:
                        for past_word in range(dec_word_index,dec_capture_len,1):
                            for rej in rej_list:
                                if rej in dec_capture[past_word]:
                                    decision=0
                                    got_dec=1
                                    break
                            for acc in acc_list:
                                if got_dec==0:
                                    if acc in dec_capture[past_word]:
                                        decision=1
                                        got_dec=1
                                else:
                                    break
                            if got_dec==1:
                                break
            if got_dec==1:
                break
    except Exception:
        print('Decision Error')
        print(traceback.format_exc())
        decision='-'
    dataf={'case title':case_title, 'judges name(s)':judges_names, 'date of judgment':case_date, 'citation':citation,'issues':issues,'decision':decision, 'cited cases':str(cited_dict)}
    df_temp=pd.DataFrame(data=dataf,index=[0])
    df=pd.concat([df,df_temp],ignore_index=True)
    no_of_links+=1
    if no_of_links%100==0:
        print('Saving...',no_of_links)
        df_save=df.iloc[-100:]
        df_save.to_csv('judge_database.csv',mode='a',header=False)      
df.to_csv('final_judge_database.csv',mode='w',header=True)


    
    

In [ ]:

# extract date, <--- from title get case name and date 
# case title, <--- from title get case name and date
# judges names, <---(3) options: Check for bench (or coram(?)); check for pre_2; print the ending of the doc
# citation, <--- check for equivalent citations; use pre_1 to get case no.
# text(?), 
# issues(Sections), <-- find using <a> tags
# dictionary of cited cases(name:score) <--- search <a> tags for ' v. ' and assign score
# create link_parser
# create data_scraper
# create while loop to query database